In [1]:
import tensorflow as tf

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import shutil
import sys

In [2]:
tf.__version__

'1.5.0'

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
from datetime import timedelta
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import log_loss,auc,classification_report,roc_auc_score, roc_curve
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, label_binarize
from sklearn import tree
import graphviz

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', None)
%matplotlib inline

data_path = str(Path(os.getcwd())) + "/data/"

def write_to_pickle(dataframe, name):
    dataframe.to_pickle(data_path + name + ".pickle")
def read_from_pickle(name): 
    return pd.read_pickle(data_path + name + ".pickle")

def binning(col, cut_points, labels=None):
    #Define min and max values:
    minval = col.min()
    maxval = col.max()

    #create list by adding min and max to cut_points
    break_points = [minval] + cut_points + [maxval]

    # if no labels provided, use default labels 0 ... (n-1)
    if not labels:
        labels = range(len(cut_points)+1)

    #Binning using cut function of pandas
    colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
    return colBin

# Process data into dummies dataframe

In [4]:
df_amelia = pd.read_csv('data/imputed_dataset_no_censoring_26022018_Amelia1.csv')

In [5]:
df_amelia.drop("Unnamed: 0", axis = 1, inplace=True)

In [6]:
labels = ["1.5year","4years","more"]
cut_points = [500,1500]

#labels = ["3_months","6_months","9_months","12_months","15_months","18_months","2_years","3_years","4_years","5_years","10_years","10_plus_years"]
#cut_points = [90,180,270,360,450,540,720,1095,1460,1825,3650]
df_amelia.loc[:,"life_expectancy_bin"] = binning(df_amelia.life_expectancy, cut_points, labels)
print(pd.value_counts(df_amelia.life_expectancy_bin, sort=False))

1.5year    921
4years     769
more       396
Name: life_expectancy_bin, dtype: int64


In [7]:
df_amelia.drop("life_expectancy", axis=1, inplace =True)

In [8]:
df_tensorflow = df_amelia.copy(deep=True)
X_train, X_test =train_test_split(df_tensorflow, train_size=0.8, test_size=0.2, random_state=1332)

print(X_train.shape, X_test.shape)

(1668, 22) (418, 22)


In [9]:
X_train.head(2)

,Gender,Tumor_type,Tumor_grade,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch10Q,Gene_Ch9P,Tumor_Location,Tumor_Position,Surgery_type,Age_surgery,IDH,TERT,IDH_TERT,X1p19q_codel,IK,life_expectancy_bin
1650,M,GBM,4,NC,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,NORMAL,Maintien,PERTE,PERTE,A,D,biopsie,61,wt,mutant,3,0,68.175581,1.5year
397,M,gliome,3,ALTERE,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,Maintien,NON PERTE,NON PERTE,A,G,exérèse,34,mutant,wt,3,0,70.000000,4years


In [10]:
# Write to CSV to not fuck up tensorflow pipeline

X_train.to_csv('data/amelia_train',index=False,columns=df_tensorflow.columns)
X_test.to_csv('data/amelia_test',index=False,columns=df_tensorflow.columns)

*** Until here all is needed for tensorflow ***
---------------------
*** DO NOT RUN UNTIL THE NEXT LINE ***

In [9]:
le_dict = dict() # Initialise an empty dictionary to keep all LabelEncoders
df_categories = df_amelia.copy(deep=True) 
# Loop over attributes by excluding the continuous oness
for column in df_categories.drop(['Age_surgery', 'life_expectancy', 'Tumor_grade','IDH_TERT','IK'], axis=1):  
    le = LabelEncoder().fit(df_categories[column]) # Initialise the LabelEncoder and fit
    df_categories[column] = le.transform(df_categories[column]) # Transform data and save in credit_clean DataFrame
    le_dict[column] = le # Store the LabelEncdoer in dictionary

In [10]:
df = df_amelia.copy(deep=True)
non_dummy_cols = ['Tumor_grade','IDH_TERT','life_expectancy','life_expectancy_bin','Gender','IK','Age_surgery']
dummy_cols = list(set(df.columns) - set(non_dummy_cols))

df = pd.get_dummies(df,columns=dummy_cols)

df.Gender.replace(to_replace={'M':1, 'F':0},inplace=True)
df.head(2)

,Gender,Tumor_grade,Age_surgery,life_expectancy,IDH_TERT,IK,life_expectancy_bin,TERT_mutant,TERT_wt,Gene_P53_ALTERE,Gene_P53_NC,Gene_P53_NORMAL,Gene_Mgmt_METHYLE,Gene_Mgmt_NON METHYLE,Tumor_type_GBM,Tumor_type_astrocytome,Tumor_type_astrocytome gémistocytique (=A2),Tumor_type_astrocytome pilocytique,Tumor_type_gangliogliome,Tumor_type_gliome,Tumor_type_oligo,Gene_Mdm2_AMPLIFIE,Gene_Mdm2_NORMAL,Gene_Ch10Q_NON PERTE,Gene_Ch10Q_PERTE,Gene_P16_DELETE,Gene_P16_NORMAL,Tumor_Location_A,Tumor_Location_B,Tumor_Location_C,Tumor_Location_N,Tumor_Location_autre,Gene_Egfr_AMPLIFIE,Gene_Egfr_NORMAL,X1p19q_codel_0,X1p19q_codel_1,IDH_NC,IDH_mutant,IDH_wt,Surgery_type_aucune,Surgery_type_biopsie,Surgery_type_exérèse,Surgery_type_partielle,Gene_Cdk4_AMPLIFIE,Gene_Cdk4_NORMAL,Tumor_Position_D,Tumor_Position_G,Tumor_Position_bilatéral,Tumor_Position_médian,Tumor_Position_non précisé,Gene_Ihc_Atrx_Extinction diffuse,Gene_Ihc_Atrx_Maintien,Gene_Ihc_Atrx_extinction,Gene_Ch9P_GAIN,Gene_Ch9P_NON PERTE,Gene_Ch9P_PERTE
0,1,3,42,30,3,90.512838,1.5year,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0
1,1,2,28,2069,2,90.596534,more,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0


In [8]:
df_categories.head(2)

,Gender,Tumor_type,Tumor_grade,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch10Q,Gene_Ch9P,Tumor_Location,Tumor_Position,Surgery_type,Age_surgery,life_expectancy,IDH,TERT,IDH_TERT,X1p19q_codel,IK,life_expectancy_bin
0,1,5,3,1,0,1,1,1,0,1,1,1,0,1,3,42,30,1,1,3,0,90.512838,0
1,1,1,2,1,1,1,1,1,1,2,0,1,3,4,2,28,2069,0,1,2,0,90.596534,2


In [ ]:
for column in df_amelia:
    unique_vals = np.unique(df_amelia[column])
    nr_vals = len(unique_vals)
    if nr_vals < 20:
        print('Number of values for attribute {}: {} -- {}'.format(column, nr_vals, unique_vals))
    else:
        print('Number of values for attribute {}: {}'.format(column, nr_vals))

_______________________________

In [9]:
df_amelia.columns

Index(['Gender', 'Tumor_type', 'Tumor_grade', 'Gene_P53', 'Gene_Mgmt',
       'Gene_Egfr', 'Gene_Mdm2', 'Gene_Cdk4', 'Gene_P16', 'Gene_Ihc_Atrx',
       'Gene_Ch10Q', 'Gene_Ch9P', 'Tumor_Location', 'Tumor_Position',
       'Surgery_type', 'Age_surgery', 'IDH', 'TERT', 'IDH_TERT',
       'X1p19q_codel', 'IK', 'life_expectancy_bin'],
      dtype='object')

In [11]:
len(df_tensorflow.columns)

22

# Apply Tensorflow Magic using the Wide and Deep method

In [12]:
flags = tf.app.flags

In [13]:
flags.DEFINE_string("train_data", "data/amelia_train", "Path to Training dataset")
flags.DEFINE_string("test_data", "data/amelia_test", "Path to Test dataset")
flags.DEFINE_integer("batch_size", 3, "Training batch size")
flags.DEFINE_string("model_dir", "data/amelia_model", "Folder that contains the model")
flags.DEFINE_integer("epochs", 40, "Number of training Epochs")
flags.DEFINE_integer("epochs_per_eval", 2, "Epochs per evaluation")
FLAGS = flags.FLAGS

In [14]:
# Arbitrary split..
_NUM_EXAMPLES = {
    'train': 1300,
    'validation': 368,
}

In [23]:
_CSV_COLUMNS = df_amelia.columns

_CSV_COLUMN_DEFAULTS = [[''], [''], [0], [''], [''], [''], [''], [''], [''],
                        [''], [''], [''], [''], [''], [''],[0],[''],[''],[0],[0],[0.0],['']]

# Does not include prediction label
def build_model_columns():
    """Builds a set of wide and deep feature columns."""
    # Continuous columns
    grade = tf.feature_column.numeric_column('Tumor_grade')
    age = tf.feature_column.numeric_column('Age_surgery')
    #survival = tf.feature_column.numeric_column('life_expectancy')
    idh_tert = tf.feature_column.numeric_column('IDH_TERT')
    ik = tf.feature_column.numeric_column('IK')
    codel = tf.feature_column.numeric_column('X1p19q_codel')
    
    # Categorical 
    gender = tf.feature_column.categorical_column_with_vocabulary_list(
      'Gender', ['M','F'])

    tumor_type = tf.feature_column.categorical_column_with_vocabulary_list(
      'Tumor_type', ['GBM' 'astrocytome' 'astrocytome gémistocytique (=A2)'
         'astrocytome pilocytique' 'gangliogliome' 'gliome' 'oligo'])
    
    gene_p53 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Gene_P53', ['ALTERE' 'NC' 'NORMAL'])
    
    gene_mgmt = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Mgmt',['METHYLE' 'NON METHYLE'])
    
    gene_egfr = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Egfr',['AMPLIFIE' 'NORMAL'])
    
    gene_mdm2 = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Mdm2',['AMPLIFIE' 'NORMAL'])
    
    gene_cdk4 = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Cdk4',['AMPLIFIE' 'NORMAL'])
    
    gene_p16 = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_P16',['DELETE','NORMAL'])
    
    gene_ihc_atrx = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Ihc_Atrx',['Extinction diffuse' 'Maintien' 'extinction'])
    
    gene_ch10q = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Ch10Q',['NON PERTE' 'PERTE'])
    
    gene_ch9p = tf.feature_column.categorical_column_with_vocabulary_list(
        'Gene_Ch9P',['GAIN' 'NON PERTE' 'PERTE'])
    
    tumor_location = tf.feature_column.categorical_column_with_vocabulary_list(
        'Tumor_Location',['A' 'B' 'C' 'N' 'autre'])
    
    surgery_type = tf.feature_column.categorical_column_with_vocabulary_list(
        'Surgery_type',['aucune' 'biopsie' 'exérèse' 'partielle'])
    
    idh = tf.feature_column.categorical_column_with_vocabulary_list(
        'IDH',['NC' 'mutant' 'wt'])
    
    tert = tf.feature_column.categorical_column_with_vocabulary_list(
        'TERT', ['mutant' 'wt'])
    
    # THE NEXT ONE IS THE PREDICTOR Y 
    #survival = tf.feature_column.categorical_column_with_vocabulary_list(
     #   'life_expectancy_bin', ['1.5year' '4years' 'more'])
    
  # To show an example of hashing 
  # When you don't know all the categories you might encounter for a certain attribute
  #survival_binned = tf.feature_column.categorical_column_with_hash_bucket(
      #'occupation', hash_bucket_size=1000)

  # Transformations - this should be Y predictor
    #survival_binned = tf.feature_column.bucketized_column(
     # survival, boundaries=[500,1500])

  # Wide columns and deep columns.
  # Categorical ones 
    base_columns =[gender, tumor_type, gene_p53, gene_mgmt, gene_egfr, gene_mdm2, gene_cdk4, gene_p16, 
      gene_ihc_atrx, gene_ch10q, gene_ch9p, tumor_location, surgery_type, idh, tert]

  # This is where a doctor's wisdom or some smart exploratory data analysis will help us
  # More information on how crossed columns work see linear model tutorial 
  # Essential part is, it takes into account combinations of these attributes so you don't miss important correlations  
  # Arguably, that could lead to a worse model than not creating these. 
    crossed_columns = [
      tf.feature_column.crossed_column(
          ['IDH', 'TERT'], hash_bucket_size=1000),
      tf.feature_column.crossed_column(
          ['Surgery_type', 'Tumor_type'], hash_bucket_size=1000),
    ]

    wide_columns = base_columns + crossed_columns

  # Linear/continuous cols, indicator transforms the categories using one-hot-encoding aka dummies
  # Should add more categories to the deep model
    deep_columns = [
      grade,
      age,
      #survival = tf.feature_column.numeric_column('life_expectancy')
      idh_tert,
      ik, 
      codel,
      #tf.feature_column.indicator_column(tumor_type),
      #tf.feature_column.indicator_column(tumor_location),
      #tf.feature_column.indicator_column(gender),
      #tf.feature_column.indicator_column(surgery_type),
      #tf.feature_column.indicator_column(idh),
      #tf.feature_column.indicator_column(tert),    
  ]

    return wide_columns, deep_columns

## Wide (categorical) and Deep (continuous) estimator 

In [16]:
# Below definition runs for only wide or only deep as well 
def build_estimator(model_dir, model_type):
    """Build an estimator appropriate for the given model type."""
    wide_columns, deep_columns = build_model_columns()
    # Should change this so it takes less time 
    hidden_units = [100, 75, 50]

    # Create a tf.estimator.RunConfig to ensure the model is run on CPU, which
    # trains faster than GPU for this model.
    #run_config = tf.estimator.RunConfig().replace(
    #session_config=tf.ConfigProto(device_count={'GPU': 0}))

    '''if model_type == 'wide':
    return tf.estimator.LinearClassifier(
        model_dir=model_dir,
        feature_columns=wide_columns,
        config=run_config)
  elif model_type == 'deep':
    return tf.estimator.DNNClassifier(
        model_dir=model_dir,
        feature_columns=deep_columns,
        hidden_units=hidden_units,
        config=run_config)
  else:'''
    return tf.estimator.DNNLinearCombinedClassifier(
        model_dir=FLAGS.model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units)
        #config=run_config)


In [20]:
# Below methods expects the dataset, train or test, num of epochs, and the rest is obvious 
def input_fn(data_file, num_epochs, shuffle, batch_size):
    """Generate an input function for the Estimator."""
    assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)
    def parse_csv(value):
        print('Parsing', data_file)
        columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
        features = dict(zip(_CSV_COLUMNS, columns))
        labels = features.pop('life_expectancy_bin')
        
        #l = tf.feature_column.numeric_column(labels)
        # ['1.5year' '4years' 'more']
        
        return features, tf.equal(labels,'more')
  # Extract lines from input files using the Dataset API.
    dataset = tf.data.TextLineDataset(data_file).skip(1)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

    dataset = dataset.map(parse_csv, num_parallel_calls=5)

    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)
    return dataset


In [21]:
def main(self):
    #tf.reset_default_graph()
    # Clean up the model directory if present
    shutil.rmtree(FLAGS.model_dir, ignore_errors=True)
    model = build_estimator(FLAGS.model_dir,None)

    # Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
    for n in range(FLAGS.epochs // FLAGS.epochs_per_eval):
        model.train(input_fn=lambda: input_fn(FLAGS.train_data, FLAGS.epochs_per_eval, True, FLAGS.batch_size))

        results = model.evaluate(input_fn=lambda: input_fn(
            FLAGS.test_data, 1, False, FLAGS.batch_size))

        # Display evaluation metrics
        print('Results at epoch', (n + 1) * FLAGS.epochs_per_eval)
        print('-' * 60)

        for key in sorted(results):
            print('%s: %s' % (key, results[key]))

In [24]:
tf.logging.set_verbosity(tf.logging.INFO)
if __name__ == '__main__':
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'data/amelia_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x125409438>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Parsing data/amelia_train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into data/amelia_model/model.ckpt.
INFO:tensorflow:loss = 0.4667895, step = 1
INFO:tensorflow:global_step/sec: 118.209
INFO:tensorflow:loss = 0.17844778, step = 101 (0.846 sec)
INFO:tensorflow:global_step/sec: 228.908
INFO:tensorflow:loss = 0.5356613, step = 201 (0.437 sec)
INFO:tensorflow:global_step/sec: 256.

SystemExit: 

/Users/patricfulop/anaconda3/envs/cancer/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
